
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y3", label="oxigenio")

/tmp/ipykernel_10026/160627683.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-24 00:48:36.717291: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 00:48:36.777093: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 00:48:36.777141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 00:48:36.778590: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 00:48:36.786820: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 00:48:36.787925: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[5], [10], [15], [20], [25], [30], [35], [40], [45]],
            regularizers=[0.01, 0.1],
            learning_rate=[0.1])

Testando combinacao1: Hidden Size=[5], regularizer=0.01, learning_rate=0.1
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  6.1924269580003966
Stopped at epoch:  0
1/1 [==============================] - 0s 27ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.05626548300006107
Stopped at epoch:  0
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.04011602700029471
Stopped at epoch:  0
1/1 [==============================] - 0s 40ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.056815402999745857
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.04533343799994327
Stopped at epoch:  0
1/1 [==============================] - 0s 24ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.045960892000039166
Stopped at epoch:  0
1/1 [==============================] - 0s 26ms/step
+++++++++++ [1_0] | 7 ++++++++

2024-03-24 00:50:46.378100: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


1/1 [==============================] - 0s 37ms/step
+++++++++++ [2_0] | 24 ++++++++++++++++++
Elapsed time:  0.05267625099986617
Stopped at epoch:  0
1/1 [==============================] - 0s 22ms/step
+++++++++++ [2_0] | 25 ++++++++++++++++++
Elapsed time:  0.051840150999851176
Stopped at epoch:  0
1/1 [==============================] - 0s 21ms/step
+++++++++++ [2_1] | 1 ++++++++++++++++++
Elapsed time:  0.4074559529999533
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [2_1] | 2 ++++++++++++++++++
Elapsed time:  0.3004076420002093
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [2_1] | 3 ++++++++++++++++++
Elapsed time:  0.8791958540004998
Stopped at epoch:  15
1/1 [==============================] - 0s 23ms/step
+++++++++++ [2_1] | 4 ++++++++++++++++++
Elapsed time:  0.6768109730001015
Stopped at epoch:  11
1/1 [==============================] - 0s 26ms/step
+++++++++++ [2_1] | 5 ++++++++++++++++++
Elapsed time

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_3_0_0,0.835891,-0.385206,0.039609,-0.085417,0.181620,0.208889,0.362428,0.281143
model_3_0_1,0.842873,-0.362753,0.119557,-0.023681,0.173893,0.205503,0.332257,0.265152
model_3_0_20,0.845375,-1.418352,0.360413,-0.183906,0.171124,0.364687,0.241364,0.306653
model_3_0_21,0.845981,-1.403405,0.363538,-0.177157,0.170454,0.362433,0.240185,0.304905
model_3_0_19,0.846218,-1.376968,0.359229,-0.171963,0.170192,0.358447,0.241811,0.303559
model_3_0_18,0.847773,-1.298182,0.363836,-0.144520,0.168470,0.346566,0.240072,0.296451
model_3_0_23,0.848477,-1.315749,0.368310,-0.146867,0.167691,0.349215,0.238384,0.297059
model_3_0_24,0.848674,-1.318268,0.372930,-0.144476,0.167473,0.349595,0.236640,0.296440
model_3_0_17,0.848690,-1.243514,0.358711,-0.131182,0.167456,0.338322,0.242006,0.292997
model_3_0_22,0.849046,-1.270646,0.364703,-0.135439,0.167062,0.342413,0.239745,0.294099


DataFrame salvo em ../results/metrics_3_0
+++++++++++ [3_1] | 1 ++++++++++++++++++
Elapsed time:  0.4229189489997225
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [3_1] | 2 ++++++++++++++++++
Elapsed time:  0.5404385680003543
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [3_1] | 3 ++++++++++++++++++
Elapsed time:  0.4507824329994037
Stopped at epoch:  6
1/1 [==============================] - 0s 41ms/step
+++++++++++ [3_1] | 4 ++++++++++++++++++
Elapsed time:  0.4236939969996456
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [3_1] | 5 ++++++++++++++++++
Elapsed time:  0.5046958399998402
Stopped at epoch:  6
1/1 [==============================] - 0s 36ms/step
+++++++++++ [3_1] | 6 ++++++++++++++++++
Elapsed time:  0.4162200679993475
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [3_1] | 7 ++++++++++++++++++
Elapsed time:  0.480081357000

...

DataFrame salvo em ../results/metrics_18_7
+++++++++++ [18_8] | 1 ++++++++++++++++++
Elapsed time:  0.19342661100017722
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_8] | 2 ++++++++++++++++++
Elapsed time:  0.18399218400008976
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [18_8] | 3 ++++++++++++++++++
Elapsed time:  0.2004338850001659
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [18_8] | 4 ++++++++++++++++++
Elapsed time:  0.20231065899861278
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_8] | 5 ++++++++++++++++++
Elapsed time:  0.19828579400018498
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_8] | 6 ++++++++++++++++++
Elapsed time:  0.19356229699951655
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [18_8] | 7 ++++++++++++++++++
Elapsed time:  0

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_18_8_4,0.800740,0.262532,0.369686,0.335425,0.220522,0.337167,0.671244,0.494380
model_18_8_3,0.839973,0.379071,0.603756,0.531029,0.177103,0.283886,0.421975,0.348869
model_18_8_2,0.863287,0.502139,0.750512,0.669938,0.151301,0.227619,0.265689,0.245534
model_18_8_1,0.882719,0.547372,0.953674,0.821519,0.129795,0.206939,0.049334,0.132772
model_18_8_0,0.883914,0.561176,0.957722,0.828738,0.128473,0.200628,0.045024,0.127402


DataFrame salvo em ../results/metrics_18_8
+++++++++++ [18_9] | 1 ++++++++++++++++++
Elapsed time:  0.19521218499903625
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_9] | 2 ++++++++++++++++++
Elapsed time:  0.2130982149992633
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [18_9] | 3 ++++++++++++++++++
Elapsed time:  0.28768387799937045
Stopped at epoch:  9
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_9] | 4 ++++++++++++++++++
Elapsed time:  0.22227168999961577
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_9] | 5 ++++++++++++++++++
Elapsed time:  0.20215895699948305
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [18_9] | 6 ++++++++++++++++++
Elapsed time:  0.19547685999896203
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [18_9] | 7 ++++++++++++++++++
Elapsed time:  0

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_18_9_11,0.800178,0.171866,-1.120786,-0.424970,0.221144,0.154084,0.877356,0.494447
model_18_9_10,0.801134,0.179011,-1.106258,-0.414791,0.220086,0.152755,0.871346,0.490915
model_18_9_9,0.801851,0.185032,-1.095460,-0.407024,0.219293,0.151634,0.866879,0.488220
model_18_9_8,0.803051,0.187476,-1.073516,-0.394018,0.217965,0.151180,0.857800,0.483707
model_18_9_4,0.804035,0.102198,-0.913967,-0.328712,0.216876,0.167047,0.791796,0.461046
model_18_9_7,0.804521,0.185116,-1.043579,-0.377893,0.216338,0.151619,0.845416,0.478112
model_18_9_6,0.805446,0.172941,-1.016846,-0.366350,0.215314,0.153884,0.834356,0.474106
model_18_9_0,0.807037,0.207821,0.622536,0.563337,0.213553,0.147394,0.156155,0.151517
model_18_9_5,0.807833,0.166472,-0.955518,-0.333778,0.212672,0.155088,0.808986,0.462804
model_18_9_3,0.838825,0.337060,-0.334488,0.063081,0.178373,0.123348,0.552069,0.325099


DataFrame salvo em ../results/metrics_18_9
